### Import

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### 정부 사업

In [39]:
page_num = 1
url = f"https://www.msit.go.kr/bbs/list.do?sCode=user&mId=129&mPid=224&pageIndex={page_num}&bbsSeqNo=100&nttSeqNo=&searchOpt=ALL&searchTxt="
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

div = soup.find_all('div',class_="toggle", limit=10)  

In [61]:
div[0]

<div class="toggle">
<a href="javascript:;" onclick="fn_detail(3178865);" title="상세보기">
<div class="num" id="td_NO_0"></div>
<div class="txt">
<p class="title" id="td_NTT_SJ_0"></p>
<div class="meta">
<dl class="tit_con">
<dt class="tit"> 노출내용</dt>
<dd class="con" id="td_NTT_CN2_0"></dd>
</dl>
<dl class="tit_con">
<dt class="tit"> 부서</dt>
<dd class="con" id="td_CHRG_DEPT_NM_0"></dd>
</dl>
<dl class="tit_con">
<dt class="tit"> 담당자</dt>
<dd class="con" id="td_NTCR_0"></dd>
</dl>
<dl class="tit_con">
<dt class="tit"> 연락처</dt>
<dd class="con" id="td_TELNO_0"></dd>
</dl>
</div>
</div>
<div class="file" id="td_FILE_0"></div>
<div class="date" id="td_REG_DT_0"></div>
</a>
</div>

In [2]:
def extractListNum(divs):
    linkIDs = []
    for div in divs:
        divText = str(div)
        staIdx = divText.find('(')
        endIdx = divText.find(')')
        linkID = divText[staIdx+1:endIdx]

        linkIDs.append(linkID)
    return linkIDs

def extractTitle(list_num):
    url = f"https://www.msit.go.kr/bbs/view.do?sCode=user&mId=129&mPid=224&pageIndex=1&bbsSeqNo=100&nttSeqNo={list_num}&searchOpt=ALL&searchTxt="
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")

    div = soup.find_all('div',class_="view_head", limit=1)  
    divText = str(div)
    staIdx = divText.find('<h2>')
    endIdx = divText.find('</h2>')
    divTitle = divText[staIdx+4:endIdx]

    return divTitle

### Load CSV

In [19]:
businessDb = pd.DataFrame(columns=["link", "title", "category", "part"])
#businessDb = pd.read_csv("ministryOfScienceAndIct_businessAnnouncement.csv")
businessDb

,link,title,category,part


In [11]:
page_num = 1
main_url = f"https://www.msit.go.kr/bbs/list.do?sCode=user&mId=129&mPid=224&pageIndex={page_num}&bbsSeqNo=100&nttSeqNo=&searchOpt=ALL&searchTxt="

page = requests.get(main_url)
soup = BeautifulSoup(page.content, "html.parser")

div = soup.find_all('div',class_="toggle", limit=10)  

urlNumList = extractListNum(div)
title = []
link = []
part = []
category = []
for urlNum in urlNumList:
    title.append(extractTitle(urlNum))
    link.append(f"https://www.msit.go.kr/bbs/view.do?sCode=user&mId=129&mPid=224&pageIndex=1&bbsSeqNo=100&nttSeqNo={urlNum}&searchOpt=ALL&searchTxt=")
    part.append("ministry of science and ict")
    category.append("")##미구현
addTuples = pd.DataFrame({"link": link, "title" : title, "category" : category, "part" : part})
afterConcat = pd.concat([businessDb,addTuples], ignore_index=True)

In [20]:
for i in range(1,31,1): # 1 page ~ 30 page
    page_num = i
    main_url = f"https://www.msit.go.kr/bbs/list.do?sCode=user&mId=129&mPid=224&pageIndex={page_num}&bbsSeqNo=100&nttSeqNo=&searchOpt=ALL&searchTxt="

    page = requests.get(main_url)
    soup = BeautifulSoup(page.content, "html.parser")   

    div = soup.find_all('div',class_="toggle", limit=10)  

    urlNumList = extractListNum(div)
    title = []
    link = []
    part = []
    category = []
    for urlNum in urlNumList:
        title.append(extractTitle(urlNum))
        link.append(f"https://www.msit.go.kr/bbs/view.do?sCode=user&mId=129&mPid=224&pageIndex=1&bbsSeqNo=100&nttSeqNo={urlNum}&searchOpt=ALL&searchTxt=")
        part.append("ministry of science and ict")
        category.append("")##미구현
    addTuples = pd.DataFrame({"link": link, "title" : title, "category" : category, "part" : part})
    businessDb = pd.concat([businessDb,addTuples], ignore_index=True)

### Save CSV

In [21]:
businessDb

,link,title,category,part
0,https://www.msit.go.kr/bbs/view.do?sCode=user&...,2024년도 고수요 데이터 확충 사업 신규과제 공고,,ministry of science and ict
1,https://www.msit.go.kr/bbs/view.do?sCode=user&...,정책연구용역(부가통신서비스 제도 개선 방안 연구) 입찰공고,,ministry of science and ict
2,https://www.msit.go.kr/bbs/view.do?sCode=user&...,정책연구용역(전략적 경쟁시대 혁신 기반의 미래 정부기능 탐색 연구) 입찰공고,,ministry of science and ict
3,https://www.msit.go.kr/bbs/view.do?sCode=user&...,정책연구용역(공공SW사업 중 신기술 과업의 적정 대가 산정방안에 관한 연구) 입찰공고,,ministry of science and ict
4,https://www.msit.go.kr/bbs/view.do?sCode=user&...,정책연구용역(과학기술분야 연구데이터 개방 및 활용을 위한 운영·관리 제도화 방안 연...,,ministry of science and ict
...,...,...,...,...
295,https://www.msit.go.kr/bbs/view.do?sCode=user&...,2023년도 한-핀란드(BF) 공동연구사업 신규과제 선정결과 공고,,ministry of science and ict
296,https://www.msit.go.kr/bbs/view.do?sCode=user&...,2023년도 한-영국(MRC) 협력기반조성사업 신규과제 선정결과 공고,,ministry of science and ict
297,https://www.msit.go.kr/bbs/view.do?sCode=user&...,2023년도 국제과학비즈니스벨트 기획연구과제 공고,,ministry of science and ict
298,https://www.msit.go.kr/bbs/view.do?sCode=user&...,2023년도 과학기술정보통신부 「안전한국훈련」 국민체험단 모집 공고,,ministry of science and ict


In [22]:
businessDb.to_csv("ministryOfScienceAndIct_businessAnnouncement.csv",index=False)